# Problem statement

Can we infer important COVID-19 public health risk factors from outdated data? In many countries census and other survey data may be incomplete or out of date. This challenge is to develop a proof-of-concept for how machine learning can help governments more accurately map COVID-19 risk in 2020 using old data, without requiring a new costly, risky, and time-consuming on-the-ground survey.

The 2011 census gives us valuable information for determining who might be most vulnerable to COVID-19 in South Africa. However, the data is nearly 10 years old, and we expect that some key indicators will have changed in that time. Building an up-to-date map showing where the most vulnerable are located will be a key step in responding to the disease. A mapping effort like this requires bringing together many different inputs and tools. For this competition, we’re starting small. Can we infer important risk factors from more readily available data?

The task is to predict the percentage of households that fall into a particularly vulnerable bracket - large households who must leave their homes to fetch water - using 2011 South African census data. Solving this challenge will show that with machine learning it is possible to use easy-to-measure stats to identify areas most at risk even in years when census data is not collected.

The challenge can be found [here](https://zindi.africa/hackathons/south-african-covid-19-vulnerability-map/)

My team's winning solution can be found [here](https://github.com/KuriaDan/Position_2_solution_for_the__African-COVID-19__zindi_hackathon/blob/master/Position_2_(Team_SomeC)_solution_for_the___African_COVID_19__zindi_hackathon.ipynb)

In [115]:
# Importing data wrangling and machine Learning packages used.
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt

import requests
from io import StringIO 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Created links to shared files via google drive
#
train = 'https://drive.google.com/file/d/1_wLi9i-pUk6Kaizjb5i6-Pd0Vi7L1d-E/view?usp=sharing'
test = 'https://drive.google.com/file/d/1OeT53v7tZLnB71n1j4r6KFbtrouz38ej/view?usp=sharing'

# Created a function to read a csv file shared via google and return a dataframe
#
def read_csv(url):
    url = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
    csv_raw = requests.get(url).text
    csv = StringIO(csv_raw)
    df = pd.read_csv(csv)
    return df

# Creating training and testing datataframes
#
train = read_csv(train)
test = read_csv(test)


print('Shape of the train data is {} while shape of Test data is {}'.format(train.shape, test.shape))


Shape of the train data is (3174, 50) while shape of Test data is (1102, 49)


In [3]:
# Previewing the first 5 observations of the train data.
train.head()

,ward,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,9D9BEUB,13569.97801,39024.03083,1.739801,0.354806,0.004781,0.170498,0.007861,0.006796,0.023082,...,0.001723,0.027796,0,0.026727,0.524666,0.041460,0.394550,0.029031,0.010292,0.599259
1,RERH3XM,13593.88256,32879.94646,1.993381,0.262140,0.002066,0.077669,0.000597,0.000977,0.003759,...,0.041982,0.021312,0,0.015754,0.993900,0.001829,0.000854,0.000586,0.002832,0.699136
2,GJWA3BO,2698.30050,8261.71093,0.038340,0.728101,0.002292,0.051663,0.008209,0.001444,0.005089,...,0.014786,0.005827,0,0.049384,0.990007,0.003582,0.002546,0.003201,0.000663,0.972315
3,GW6VRB7,1965.96698,9069.77694,7.411006,0.613206,0.138127,0.004999,0.002704,0.000000,0.002788,...,0.003087,0.003136,0,0.013562,0.928682,0.004464,0.009854,0.054701,0.002299,0.735333
4,CN9JZJC,1852.07821,9908.84404,8.223907,0.797462,0.120830,0.060677,0.001155,0.001042,0.000508,...,0.000117,0.001471,0,0.000000,0.994072,0.000476,0.004033,0.000704,0.000714,0.646964


In [4]:
# Previewing the forst 5 observations of the test data.
test.head()

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,00A93O0,2514.04172,8037.06199,0.671962,0.061074,0.101516,0.006891,0.021287,0.015615,0.035974,...,0.000420,0.011485,0,0.063705,0.979272,0.009926,0.003224,0.002697,0.004882,0.521007
1,01V40KO,1829.71944,7494.86917,0.542857,0.390850,0.039623,0.000000,0.000000,0.000000,0.002633,...,0.000000,0.000160,0,0.000000,0.998938,0.000296,0.000301,0.000305,0.000160,0.693716
2,01ZHVW0,1721.99009,5847.43254,0.968071,0.003525,0.001431,0.000000,0.000000,0.000000,0.018546,...,0.000631,0.006504,0,0.000171,0.955895,0.012170,0.000000,0.030012,0.001923,0.933954
3,028YQJX,12388.02257,52211.66859,0.397274,0.004734,0.001068,0.000279,0.000799,0.197760,0.005227,...,0.000593,0.008330,0,0.000000,0.293629,0.694779,0.003238,0.000628,0.007726,0.921470
4,02G8W11,1754.25716,4266.57609,0.815197,0.016697,0.078571,0.008975,0.047882,0.015398,0.007264,...,0.000234,0.016846,0,0.178598,0.110861,0.054114,0.006062,0.823495,0.005470,0.996227


In [137]:
# While quickly glancing on the data, some columns appear to only have 0s. 
# Checking one such column to find the number of unique values in the column.
train.lan_13.nunique()

AttributeError: 'DataFrame' object has no attribute 'lan_13'

In [8]:
# Looping through the train data to drop those columns.

for col in train.columns:
    if train[col].nunique() == 1:
        train = train.drop(col, axis=1)

        

In [7]:
train.head()

,ward,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,lan_10,lan_11,lan_12,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,9D9BEUB,13569.97801,39024.03083,1.739801,0.354806,0.004781,0.170498,0.007861,0.006796,0.023082,...,0.000787,0.001723,0.027796,0.026727,0.524666,0.041460,0.394550,0.029031,0.010292,0.599259
1,RERH3XM,13593.88256,32879.94646,1.993381,0.262140,0.002066,0.077669,0.000597,0.000977,0.003759,...,0.005768,0.041982,0.021312,0.015754,0.993900,0.001829,0.000854,0.000586,0.002832,0.699136
2,GJWA3BO,2698.30050,8261.71093,0.038340,0.728101,0.002292,0.051663,0.008209,0.001444,0.005089,...,0.002755,0.014786,0.005827,0.049384,0.990007,0.003582,0.002546,0.003201,0.000663,0.972315
3,GW6VRB7,1965.96698,9069.77694,7.411006,0.613206,0.138127,0.004999,0.002704,0.000000,0.002788,...,0.001135,0.003087,0.003136,0.013562,0.928682,0.004464,0.009854,0.054701,0.002299,0.735333
4,CN9JZJC,1852.07821,9908.84404,8.223907,0.797462,0.120830,0.060677,0.001155,0.001042,0.000508,...,0.000236,0.000117,0.001471,0.000000,0.994072,0.000476,0.004033,0.000704,0.000714,0.646964


In [136]:
# Checking for null values
if train.isnull().sum().sum() == 0:
    print('no NaN found')

no NaN found


## EDA

In [11]:
train.target_pct_vunerable.describe()

count    3174.000000
mean       10.258947
std        12.847192
min         0.000000
25%         0.429761
50%         3.715314
75%        17.762604
max       100.000000
Name: target_pct_vunerable, dtype: float64

  ### Baseline CatBoost model.


In [139]:
# Training a baseline LGBM model
X=train.drop(["ward",'target_pct_vunerable'],axis =1)
y = train.target_pct_vunerable

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
est = LGBMRegressor().fit(x_train, y_train)
y_pred = est.predict(x_test)
np.sqrt(mean_squared_error(y_pred, y_test))


5.819721389412163

In [143]:
# Training a baseline LGBM model
X=train.drop(["ward",'target_pct_vunerable'],axis =1)
y = train.target_pct_vunerable

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=90)
est = LGBMRegressor().fit(x_train, y_train)
y_pred = est.predict(x_test)
np.sqrt(mean_squared_error(y_pred, y_test))


5.600945528134281

In [18]:
print(LGBMRegressor())

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


### Hyperparameters

> boosting_type - LGBM can run different Gradient Boosting methods, from traditional GBDT to DART(dropout) and GOSS(gradient based one side sampling)

> num_leaves - Sets the maximum number of leaves each weak learner has.

> Subsample - also called bagging fraction. specifies the percentage of rows used per tree building iteration.

> max_depth - maximum depth of a tree.

> num_iterations - number of boosting iterations. 

> Learning rate - determines the step size at each interation while moving towards a minimum of a loss function.

## Approach 1: 

### Manually tuning by trial and error.

In [55]:
# Tuning the boosting 
est = LGBMRegressor(boosting_type='dart').fit(x_train, y_train)
y_pred = est.predict(x_test)
np.sqrt(mean_squared_error(y_pred, y_test))


6.13047631213017

In [56]:
est = LGBMRegressor(boosting_type='goss').fit(x_train, y_train)
y_pred = est.predict(x_test)
np.sqrt(mean_squared_error(y_pred, y_test))


266482.3573341219

## Approach 2: 

### Manually tuning using cross validation

In [81]:
%%time
# Here, we increase the number of trees while reducing the learning rate with the same factor.
# First, we try with a five fold cross validation.

scores = cross_val_score(LGBMRegressor(n_estimators=1000, learning_rate=0.01), X, y, cv=5, scoring='neg_root_mean_squared_error')
print(scores)
print(np.mean(scores))

[-6.19706013 -6.74925307 -5.90050262 -5.89502568 -5.48076016]
-6.044520331167662
Wall time: 44.3 s


In [82]:
%%time
# we keep everything constant but change the number of folds from 5 to 7.
scores = cross_val_score(LGBMRegressor(n_estimators=1000, learning_rate=0.01), X, y, cv=7, scoring='neg_root_mean_squared_error')
print(scores)
print(np.mean(scores))

[-6.0357947  -6.21211195 -6.85421203 -6.01496085 -5.75760133 -5.42536874
 -5.64013889]
-5.991455498711916
Wall time: 45.5 s


### Automating above process with a for loop

In [144]:
np.arange(0.01, 0.1, 0.02)

array([0.01, 0.03, 0.05, 0.07, 0.09])

In [85]:
%%time
trees = [900, 1000]
etas = np.arange(0.01, 0.1, 0.02)
leaves=[25, 30, 35]
average_scores = {}

for tree in trees:
    for eta in etas:
        for leaf in leaves:
            cv_scores=cross_val_score(LGBMRegressor(n_estimators=tree, learning_rate=eta, num_leaves=leaf), X, y, cv=7, scoring='neg_root_mean_squared_error')
            average_scores[str(tree)+'_'+str(eta)+'_'+str(leaf)] = np.average(cv_scores)

                
average_scores

Wall time: 21min 11s


{'900_0.01_25': -6.024943998262722,
 '900_0.01_30': -6.0033730629250615,
 '900_0.01_35': -6.000240447936209,
 '900_0.03_25': -5.99873906147097,
 '900_0.03_30': -5.998782841644078,
 '900_0.03_35': -5.991237253591733,
 '900_0.049999999999999996_25': -5.978094133503604,
 '900_0.049999999999999996_30': -6.027061729006964,
 '900_0.049999999999999996_35': -6.042119483239447,
 '900_0.06999999999999999_25': -6.079160782611086,
 '900_0.06999999999999999_30': -6.0199674358488755,
 '900_0.06999999999999999_35': -6.042249308664408,
 '900_0.08999999999999998_25': -6.041643675170066,
 '900_0.08999999999999998_30': -6.060179108707479,
 '900_0.08999999999999998_35': -6.0499729080135305,
 '1000_0.01_25': -6.023630889176753,
 '1000_0.01_30': -5.999268020470362,
 '1000_0.01_35': -5.995237830739831,
 '1000_0.03_25': -5.997244148162144,
 '1000_0.03_30': -5.999038051101979,
 '1000_0.03_35': -5.990207892424167,
 '1000_0.049999999999999996_25': -5.977333086455718,
 '1000_0.049999999999999996_30': -6.027365960

## Approach 3: GridSearchCV

In [145]:
dir(est)

['_Booster',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_doc',
 '_best_iteration',
 '_best_score',
 '_classes',
 '_estimator_type',
 '_evals_result',
 '_fobj',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_n_classes',
 '_n_features',
 '_objective',
 '_other_params',
 'best_iteration_',
 'best_score_',
 'booster_',
 'boosting_type',
 'class_weight',
 'colsample_bytree',
 'evals_result_',
 'feature_importances_',
 'fit',
 'get_params',
 'importance_type',
 'learning_rate',
 'max_depth',
 'min_child_samples',
 'min_child_weight',
 'min_split_gain',
 'n_estimators',
 'n_features_',
 'n_jobs',
 'num_leaves',
 'objective',


In [102]:
%%time
# Gridsearch does exactly the same thing as the for loop above but in a single line of code.

est = GridSearchCV(LGBMRegressor(), {
    'n_estimators': [900, 1000],
    'learning_rate': np.arange(0.01, 0.1, 0.02),
    'num_leaves': [25,30,35]
}, cv=7, scoring='neg_root_mean_squared_error')
est.fit(X, y)
est.cv_results_

Wall time: 20min 14s


{'mean_fit_time': array([4.39750886, 5.28376647, 6.2964249 , 5.3736215 , 5.97589145,
        9.8461131 , 5.03412444, 5.09619073, 6.07820521, 5.50162411,
        6.17924782, 7.33211609, 4.54206879, 4.99899602, 6.09905366,
        4.67715536, 5.47795354, 6.31255528, 4.1936357 , 6.02828632,
        8.22946838, 4.83653658, 5.6129539 , 6.36149624, 4.21140259,
        4.96741959, 6.15578423, 4.74268835, 5.51626086, 6.43970701]),
 'std_fit_time': array([0.24930933, 0.47002929, 0.51529774, 0.57432461, 0.27951155,
        0.74424484, 1.12396682, 0.28242381, 0.35414954, 0.5062751 ,
        0.43130033, 0.45773767, 0.21134533, 0.22771023, 0.3770536 ,
        0.09468852, 0.1360997 , 0.17045921, 0.09287023, 1.18877976,
        1.35297405, 0.43341485, 0.34997158, 0.13791134, 0.09008101,
        0.15058712, 0.51151248, 0.05894681, 0.11312743, 0.1669518 ]),
 'mean_score_time': array([0.01957556, 0.0297151 , 0.02857229, 0.02971567, 0.0274178 ,
        0.0306687 , 0.02672124, 0.02857137, 0.03428667, 0.03

In [103]:
results=pd.DataFrame(est.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,mean_test_score,std_test_score,rank_test_score
0,4.397509,0.249309,0.019576,0.003847,0.01,900,25,"{'learning_rate': 0.01, 'n_estimators': 900, '...",-6.092916,-6.205463,-6.915455,-6.102232,-5.777818,-5.443539,-5.637185,-6.024944,0.444916,16
1,5.283766,0.470029,0.029715,0.011877,0.01,900,30,"{'learning_rate': 0.01, 'n_estimators': 900, '...",-6.001506,-6.189596,-6.903116,-6.053886,-5.810777,-5.452851,-5.611881,-6.003373,0.437559,12
2,6.296425,0.515298,0.028572,0.008398,0.01,900,35,"{'learning_rate': 0.01, 'n_estimators': 900, '...",-6.071855,-6.272105,-6.812177,-6.048126,-5.737991,-5.431419,-5.628010,-6.000240,0.426189,11
3,5.373621,0.574325,0.029716,0.009285,0.01,1000,25,"{'learning_rate': 0.01, 'n_estimators': 1000, ...",-6.091492,-6.204183,-6.929408,-6.088594,-5.769661,-5.442899,-5.639179,-6.023631,0.449032,15
4,5.975891,0.279512,0.027418,0.003944,0.01,1000,30,"{'learning_rate': 0.01, 'n_estimators': 1000, ...",-5.998163,-6.176304,-6.897094,-6.056716,-5.808651,-5.456887,-5.601060,-5.999268,0.435860,10
5,9.846113,0.744245,0.030669,0.002841,0.01,1000,35,"{'learning_rate': 0.01, 'n_estimators': 1000, ...",-6.063558,-6.266234,-6.813757,-6.039355,-5.732210,-5.427474,-5.624078,-5.995238,0.427505,5
6,5.034124,1.123967,0.026721,0.003494,0.03,900,25,"{'learning_rate': 0.03, 'n_estimators': 900, '...",-6.013939,-6.227165,-6.933301,-5.938894,-5.765084,-5.416821,-5.695970,-5.998739,0.449490,7
7,5.096191,0.282424,0.028571,0.003957,0.03,900,30,"{'learning_rate': 0.03, 'n_estimators': 900, '...",-6.020098,-6.164811,-6.932594,-6.042346,-5.811866,-5.410380,-5.609385,-5.998783,0.452723,8
8,6.078205,0.354150,0.034287,0.008241,0.03,900,35,"{'learning_rate': 0.03, 'n_estimators': 900, '...",-6.003539,-6.110953,-6.860969,-6.055711,-5.736761,-5.458780,-5.711949,-5.991237,0.414271,4
9,5.501624,0.506275,0.031801,0.004397,0.03,1000,25,"{'learning_rate': 0.03, 'n_estimators': 1000, ...",-6.007870,-6.222329,-6.931115,-5.937093,-5.769857,-5.417119,-5.695327,-5.997244,0.448157,6


In [104]:
results[['param_n_estimators', 'param_learning_rate', 'param_num_leaves','mean_test_score']]

,param_n_estimators,param_learning_rate,param_num_leaves,mean_test_score
0,900,0.01,25,-6.024944
1,900,0.01,30,-6.003373
2,900,0.01,35,-6.000240
3,1000,0.01,25,-6.023631
4,1000,0.01,30,-5.999268
5,1000,0.01,35,-5.995238
6,900,0.03,25,-5.998739
7,900,0.03,30,-5.998783
8,900,0.03,35,-5.991237
9,1000,0.03,25,-5.997244


In [105]:
est.best_params_

{'learning_rate': 0.049999999999999996, 'n_estimators': 1000, 'num_leaves': 25}

In [106]:
est.best_score_

-5.977333086455718

## Approach 4

### Random Search

In [132]:
%%time
# Random search reduces the number of iterations by using  random combination of the hyperparameter grid.
# It's therefore inexpensive computationally.
est = RandomizedSearchCV(LGBMRegressor(), {
    'n_estimators': [900, 1000],
    'learning_rate': np.arange(0.01, 0.1, 0.02),
    'num_leaves': [25,30,35]
}, cv=7, scoring='neg_root_mean_squared_error', n_iter=8)
est.fit(X, y)
est.cv_results_

Wall time: 5min 50s


{'mean_fit_time': array([8.31200695, 6.82716094, 6.68910142, 5.91374772, 5.7614413 ,
        4.35420694, 6.46812187, 4.51092979]),
 'std_fit_time': array([1.43794783, 0.49508203, 0.46940929, 0.46553177, 0.14452858,
        0.09432378, 0.10913395, 0.06095054]),
 'mean_score_time': array([0.04060701, 0.04046375, 0.03889765, 0.02878199, 0.0286384 ,
        0.02379496, 0.0287822 , 0.02208533]),
 'std_score_time': array([0.01611359, 0.0131857 , 0.0107679 , 0.00187963, 0.00254074,
        0.00098704, 0.0030359 , 0.00063705]),
 'param_num_leaves': masked_array(data=[30, 30, 35, 30, 35, 25, 35, 25],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 1000, 1000, 1000, 900, 900, 1000, 900],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.01, 0.03, 

In [110]:
pd.DataFrame(est.cv_results_)[['param_n_estimators', 'param_learning_rate', 'param_num_leaves','mean_test_score']]

,param_n_estimators,param_learning_rate,param_num_leaves,mean_test_score
0,1000,0.03,35,-5.990208
1,900,0.07,25,-6.079161
2,1000,0.07,25,-6.080088
3,1000,0.09,35,-6.050062
4,1000,0.01,35,-5.995238
5,900,0.03,30,-5.998783
6,900,0.09,25,-6.041644
7,900,0.05,25,-5.978094
8,1000,0.05,25,-5.977333
9,900,0.05,30,-6.027062


In [112]:
est.best_params_

{'num_leaves': 35, 'n_estimators': 1000, 'learning_rate': 0.03}

In [113]:
est.best_score_

-5.990207892424167

## Finally...

In [120]:
# Creating the model with the best parameters using kfold cross-validation.
X=train.drop(["ward",'target_pct_vunerable'],axis =1)
y = train.target_pct_vunerable


score=[]


fold=KFold(n_splits=7)
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = LGBMRegressor(n_estimators=1000, learning_rate=0.05, number_of_leaves=25, silent=False)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)])
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    score.append(np.sqrt(mean_squared_error(y_test,preds)))


[1]	training's l2: 151.2	valid_1's l2: 161.329
[2]	training's l2: 140.155	valid_1's l2: 150.21
[3]	training's l2: 130.014	valid_1's l2: 140.005
[4]	training's l2: 120.752	valid_1's l2: 131.169
[5]	training's l2: 112.365	valid_1's l2: 123.188
[6]	training's l2: 104.736	valid_1's l2: 115.661
[7]	training's l2: 97.8194	valid_1's l2: 108.84
[8]	training's l2: 91.5163	valid_1's l2: 102.769
[9]	training's l2: 85.5958	valid_1's l2: 97.0565
[10]	training's l2: 80.2592	valid_1's l2: 91.9018
[11]	training's l2: 75.4213	valid_1's l2: 87.5075
[12]	training's l2: 70.8903	valid_1's l2: 83.3745
[13]	training's l2: 66.7821	valid_1's l2: 79.365
[14]	training's l2: 63.0291	valid_1's l2: 75.9629
[15]	training's l2: 59.5148	valid_1's l2: 72.4686
[16]	training's l2: 56.3858	valid_1's l2: 69.6038
[17]	training's l2: 53.5181	valid_1's l2: 66.9955
[18]	training's l2: 50.8492	valid_1's l2: 64.7802
[19]	training's l2: 48.2901	valid_1's l2: 62.3337
[20]	training's l2: 46.0285	valid_1's l2: 60.3501
[21]	training'

[172]	training's l2: 4.45965	valid_1's l2: 36.4824
[173]	training's l2: 4.42632	valid_1's l2: 36.4777
[174]	training's l2: 4.39052	valid_1's l2: 36.4593
[175]	training's l2: 4.35508	valid_1's l2: 36.4383
[176]	training's l2: 4.31704	valid_1's l2: 36.434
[177]	training's l2: 4.28184	valid_1's l2: 36.4222
[178]	training's l2: 4.24684	valid_1's l2: 36.3546
[179]	training's l2: 4.20963	valid_1's l2: 36.3566
[180]	training's l2: 4.18024	valid_1's l2: 36.3656
[181]	training's l2: 4.14969	valid_1's l2: 36.354
[182]	training's l2: 4.11472	valid_1's l2: 36.3658
[183]	training's l2: 4.06839	valid_1's l2: 36.3615
[184]	training's l2: 4.04139	valid_1's l2: 36.3646
[185]	training's l2: 4.00339	valid_1's l2: 36.3725
[186]	training's l2: 3.96999	valid_1's l2: 36.3981
[187]	training's l2: 3.94347	valid_1's l2: 36.3886
[188]	training's l2: 3.91318	valid_1's l2: 36.3855
[189]	training's l2: 3.87756	valid_1's l2: 36.4083
[190]	training's l2: 3.85023	valid_1's l2: 36.4063
[191]	training's l2: 3.81906	vali

[345]	training's l2: 1.37098	valid_1's l2: 36.3256
[346]	training's l2: 1.36265	valid_1's l2: 36.3202
[347]	training's l2: 1.35616	valid_1's l2: 36.3068
[348]	training's l2: 1.34965	valid_1's l2: 36.2965
[349]	training's l2: 1.34045	valid_1's l2: 36.2936
[350]	training's l2: 1.33303	valid_1's l2: 36.2944
[351]	training's l2: 1.32362	valid_1's l2: 36.2884
[352]	training's l2: 1.31513	valid_1's l2: 36.2833
[353]	training's l2: 1.3082	valid_1's l2: 36.2859
[354]	training's l2: 1.30022	valid_1's l2: 36.272
[355]	training's l2: 1.29354	valid_1's l2: 36.2657
[356]	training's l2: 1.28553	valid_1's l2: 36.2661
[357]	training's l2: 1.27862	valid_1's l2: 36.2676
[358]	training's l2: 1.27022	valid_1's l2: 36.2767
[359]	training's l2: 1.26078	valid_1's l2: 36.2786
[360]	training's l2: 1.25525	valid_1's l2: 36.2731
[361]	training's l2: 1.24848	valid_1's l2: 36.2812
[362]	training's l2: 1.24293	valid_1's l2: 36.2843
[363]	training's l2: 1.2366	valid_1's l2: 36.3064
[364]	training's l2: 1.2276	valid_

[512]	training's l2: 0.562043	valid_1's l2: 36.3107
[513]	training's l2: 0.559326	valid_1's l2: 36.3182
[514]	training's l2: 0.556427	valid_1's l2: 36.3177
[515]	training's l2: 0.553608	valid_1's l2: 36.3163
[516]	training's l2: 0.55134	valid_1's l2: 36.3144
[517]	training's l2: 0.548502	valid_1's l2: 36.3171
[518]	training's l2: 0.545863	valid_1's l2: 36.3175
[519]	training's l2: 0.5428	valid_1's l2: 36.3155
[520]	training's l2: 0.540541	valid_1's l2: 36.3145
[521]	training's l2: 0.537392	valid_1's l2: 36.3179
[522]	training's l2: 0.534917	valid_1's l2: 36.3206
[523]	training's l2: 0.532145	valid_1's l2: 36.3186
[524]	training's l2: 0.529036	valid_1's l2: 36.3214
[525]	training's l2: 0.52648	valid_1's l2: 36.3236
[526]	training's l2: 0.523763	valid_1's l2: 36.3182
[527]	training's l2: 0.521221	valid_1's l2: 36.3132
[528]	training's l2: 0.519107	valid_1's l2: 36.3123
[529]	training's l2: 0.516367	valid_1's l2: 36.3099
[530]	training's l2: 0.514019	valid_1's l2: 36.3089
[531]	training's

[687]	training's l2: 0.25638	valid_1's l2: 36.2591
[688]	training's l2: 0.255208	valid_1's l2: 36.2591
[689]	training's l2: 0.253929	valid_1's l2: 36.2627
[690]	training's l2: 0.252937	valid_1's l2: 36.2624
[691]	training's l2: 0.252065	valid_1's l2: 36.2585
[692]	training's l2: 0.250989	valid_1's l2: 36.2624
[693]	training's l2: 0.250046	valid_1's l2: 36.2637
[694]	training's l2: 0.248844	valid_1's l2: 36.2622
[695]	training's l2: 0.248035	valid_1's l2: 36.2591
[696]	training's l2: 0.246841	valid_1's l2: 36.2567
[697]	training's l2: 0.245724	valid_1's l2: 36.2542
[698]	training's l2: 0.244812	valid_1's l2: 36.2552
[699]	training's l2: 0.243657	valid_1's l2: 36.2552
[700]	training's l2: 0.242592	valid_1's l2: 36.2537
[701]	training's l2: 0.2415	valid_1's l2: 36.2532
[702]	training's l2: 0.240271	valid_1's l2: 36.2524
[703]	training's l2: 0.239142	valid_1's l2: 36.2548
[704]	training's l2: 0.238149	valid_1's l2: 36.2557
[705]	training's l2: 0.237278	valid_1's l2: 36.2572
[706]	training'

...

[201]	training's l2: 3.68556	valid_1's l2: 33.2329
[202]	training's l2: 3.65558	valid_1's l2: 33.2587
[203]	training's l2: 3.62549	valid_1's l2: 33.2123
[204]	training's l2: 3.59296	valid_1's l2: 33.2064
[205]	training's l2: 3.55882	valid_1's l2: 33.196
[206]	training's l2: 3.53704	valid_1's l2: 33.2097
[207]	training's l2: 3.50775	valid_1's l2: 33.2246
[208]	training's l2: 3.47877	valid_1's l2: 33.2252
[209]	training's l2: 3.45447	valid_1's l2: 33.2106
[210]	training's l2: 3.43619	valid_1's l2: 33.1932
[211]	training's l2: 3.40958	valid_1's l2: 33.1933
[212]	training's l2: 3.38482	valid_1's l2: 33.1885
[213]	training's l2: 3.36545	valid_1's l2: 33.1845
[214]	training's l2: 3.33976	valid_1's l2: 33.176
[215]	training's l2: 3.3075	valid_1's l2: 33.1967
[216]	training's l2: 3.2889	valid_1's l2: 33.2068
[217]	training's l2: 3.26434	valid_1's l2: 33.1736
[218]	training's l2: 3.24056	valid_1's l2: 33.1516
[219]	training's l2: 3.21612	valid_1's l2: 33.13
[220]	training's l2: 3.19399	valid_1'

[385]	training's l2: 1.11045	valid_1's l2: 32.977
[386]	training's l2: 1.10507	valid_1's l2: 32.9751
[387]	training's l2: 1.09921	valid_1's l2: 32.9721
[388]	training's l2: 1.09238	valid_1's l2: 32.9677
[389]	training's l2: 1.08561	valid_1's l2: 32.9596
[390]	training's l2: 1.08041	valid_1's l2: 32.9633
[391]	training's l2: 1.07463	valid_1's l2: 32.968
[392]	training's l2: 1.06896	valid_1's l2: 32.9707
[393]	training's l2: 1.06337	valid_1's l2: 32.9706
[394]	training's l2: 1.05702	valid_1's l2: 32.9763
[395]	training's l2: 1.051	valid_1's l2: 32.9696
[396]	training's l2: 1.04603	valid_1's l2: 32.9596
[397]	training's l2: 1.04028	valid_1's l2: 32.9631
[398]	training's l2: 1.03486	valid_1's l2: 32.9593
[399]	training's l2: 1.02895	valid_1's l2: 32.9613
[400]	training's l2: 1.0238	valid_1's l2: 32.959
[401]	training's l2: 1.01806	valid_1's l2: 32.9648
[402]	training's l2: 1.01229	valid_1's l2: 32.9596
[403]	training's l2: 1.00687	valid_1's l2: 32.9606
[404]	training's l2: 1.00169	valid_1'

[545]	training's l2: 0.483973	valid_1's l2: 32.8854
[546]	training's l2: 0.48146	valid_1's l2: 32.8863
[547]	training's l2: 0.478903	valid_1's l2: 32.8896
[548]	training's l2: 0.476583	valid_1's l2: 32.8975
[549]	training's l2: 0.474707	valid_1's l2: 32.9005
[550]	training's l2: 0.472646	valid_1's l2: 32.9067
[551]	training's l2: 0.469884	valid_1's l2: 32.9091
[552]	training's l2: 0.467447	valid_1's l2: 32.9068
[553]	training's l2: 0.465249	valid_1's l2: 32.8976
[554]	training's l2: 0.463292	valid_1's l2: 32.892
[555]	training's l2: 0.461165	valid_1's l2: 32.8864
[556]	training's l2: 0.458738	valid_1's l2: 32.8865
[557]	training's l2: 0.456362	valid_1's l2: 32.8882
[558]	training's l2: 0.454254	valid_1's l2: 32.8901
[559]	training's l2: 0.452083	valid_1's l2: 32.8863
[560]	training's l2: 0.449996	valid_1's l2: 32.8871
[561]	training's l2: 0.448098	valid_1's l2: 32.8885
[562]	training's l2: 0.446105	valid_1's l2: 32.8893
[563]	training's l2: 0.44439	valid_1's l2: 32.8887
[564]	training'

[730]	training's l2: 0.21168	valid_1's l2: 32.9191
[731]	training's l2: 0.210585	valid_1's l2: 32.9198
[732]	training's l2: 0.209691	valid_1's l2: 32.9182
[733]	training's l2: 0.208818	valid_1's l2: 32.9178
[734]	training's l2: 0.207872	valid_1's l2: 32.9164
[735]	training's l2: 0.207193	valid_1's l2: 32.9177
[736]	training's l2: 0.20622	valid_1's l2: 32.9188
[737]	training's l2: 0.205425	valid_1's l2: 32.9208
[738]	training's l2: 0.204489	valid_1's l2: 32.9198
[739]	training's l2: 0.203567	valid_1's l2: 32.9187
[740]	training's l2: 0.202805	valid_1's l2: 32.9195
[741]	training's l2: 0.201858	valid_1's l2: 32.9177
[742]	training's l2: 0.200851	valid_1's l2: 32.9174
[743]	training's l2: 0.199976	valid_1's l2: 32.9177
[744]	training's l2: 0.199109	valid_1's l2: 32.9185
[745]	training's l2: 0.198268	valid_1's l2: 32.9198
[746]	training's l2: 0.197317	valid_1's l2: 32.9138
[747]	training's l2: 0.196617	valid_1's l2: 32.9134
[748]	training's l2: 0.195734	valid_1's l2: 32.9135
[749]	training

[913]	training's l2: 0.103281	valid_1's l2: 32.8952
[914]	training's l2: 0.102892	valid_1's l2: 32.8984
[915]	training's l2: 0.102514	valid_1's l2: 32.8984
[916]	training's l2: 0.102038	valid_1's l2: 32.8939
[917]	training's l2: 0.101672	valid_1's l2: 32.8937
[918]	training's l2: 0.101248	valid_1's l2: 32.8935
[919]	training's l2: 0.100799	valid_1's l2: 32.8924
[920]	training's l2: 0.100426	valid_1's l2: 32.8908
[921]	training's l2: 0.100084	valid_1's l2: 32.8916
[922]	training's l2: 0.0997057	valid_1's l2: 32.8927
[923]	training's l2: 0.0993588	valid_1's l2: 32.8955
[924]	training's l2: 0.0990076	valid_1's l2: 32.8958
[925]	training's l2: 0.0986597	valid_1's l2: 32.8946
[926]	training's l2: 0.0982498	valid_1's l2: 32.8957
[927]	training's l2: 0.0978666	valid_1's l2: 32.8947
[928]	training's l2: 0.0974894	valid_1's l2: 32.8964
[929]	training's l2: 0.0970969	valid_1's l2: 32.8994
[930]	training's l2: 0.0967197	valid_1's l2: 32.8972
[931]	training's l2: 0.0964121	valid_1's l2: 32.8979
[9

In [121]:
score

[6.020986223436029,
 6.015411374882773,
 6.816948350833014,
 6.028448071271268,
 5.869337738861242,
 5.4905455006475865,
 5.737646484967558]

In [122]:
np.mean(score)

5.997046249271352

> Using Cross-validation and hyperparameter tuning, we were able to improve our baseline's model performance from an RMSE of 6.1 to an RMSE of 5.99.